In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [2]:
# Data Augmentation for Training
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
training_set = train_datagen.flow_from_directory(
    "Monkeypox/archive (60)/Augmented Images/Augmented Images/FOLDS_AUG/fold3_AUG/Train/",
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',  # Use 'categorical' for one-hot encoded labels
)

Found 7532 images belonging to 6 classes.


In [3]:
test_datagen = ImageDataGenerator(rescale=1./255)
validation_set = test_datagen.flow_from_directory(
    "Monkeypox/archive (60)/Augmented Images/Augmented Images/FOLDS_AUG/fold3_AUG/Train/",
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',  # Use 'categorical' for one-hot encoded labels
)

Found 7532 images belonging to 6 classes.


In [4]:
# Custom F1 Score Metric
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        # Update the precision and recall for each batch
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        # Calculate F1 score as the harmonic mean of precision and recall
        precision = self.precision.result()
        recall = self.recall.result()
        return 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())

    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()


In [5]:
# Build the Bi-LSTM Model
bi_lstm_model = tf.keras.models.Sequential()

In [6]:
# Flatten and Reshape for Bi-LSTM
bi_lstm_model.add(tf.keras.layers.Flatten(input_shape=(64, 64, 3)))
bi_lstm_model.add(tf.keras.layers.Reshape((16, -1)))  # Reshape to (time_steps, features)

C:\Users\KIIT\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [7]:
# Bidirectional LSTM Layer
bi_lstm_model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=64, activation='tanh', return_sequences=False)))

In [8]:
# Fully Connected Layers
bi_lstm_model.add(tf.keras.layers.Dense(units=128, activation='relu'))
bi_lstm_model.add(tf.keras.layers.Dense(6, activation='softmax'))  # 6 classes

In [9]:
# Compile the model with Precision, Recall, and F1 Score
bi_lstm_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',  # Use categorical_crossentropy for one-hot encoded labels
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        F1Score(name='f1_score')  # Add custom F1 score metric
    ]
)

In [10]:
# Train the model
history = bi_lstm_model.fit(
    x=training_set,
    validation_data=validation_set,
    epochs=100
)

C:\Users\KIIT\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100
236/236 ━━━━━━━━━━━━━━━━━━━━ 0s 289ms/step - accuracy: 0.3851 - f1_score: 0.1006 - loss: 1.5949 - precision: 0.4890 - recall: 0.0567

C:\Users\KIIT\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


236/236 ━━━━━━━━━━━━━━━━━━━━ 109s 404ms/step - accuracy: 0.3852 - f1_score: 0.1007 - loss: 1.5948 - precision: 0.4892 - recall: 0.0568 - val_accuracy: 0.4133 - val_f1_score: 0.1287 - val_loss: 1.5196 - val_precision: 0.6301 - val_recall: 0.0717
Epoch 2/100
236/236 ━━━━━━━━━━━━━━━━━━━━ 93s 396ms/step - accuracy: 0.4094 - f1_score: 0.1566 - loss: 1.5222 - precision: 0.5812 - recall: 0.0906 - val_accuracy: 0.4271 - val_f1_score: 0.1971 - val_loss: 1.4758 - val_precision: 0.6046 - val_recall: 0.1178
Epoch 3/100
236/236 ━━━━━━━━━━━━━━━━━━━━ 123s 524ms/step - accuracy: 0.4233 - f1_score: 0.1862 - loss: 1.4884 - precision: 0.5947 - recall: 0.1106 - val_accuracy: 0.4401 - val_f1_score: 0.2712 - val_loss: 1.4524 - val_precision: 0.5728 - val_recall: 0.1776
Epoch 4/100
236/236 ━━━━━━━━━━━━━━━━━━━━ 126s 533ms/step - accuracy: 0.4403 - f1_score: 0.2273 - loss: 1.4466 - precision: 0.5948 - recall: 0.1408 - val_accuracy: 0.4399 - val_f1_score: 0.2524 - val_loss: 1.4282 - val_precision: 0.6445 - val_

In [13]:
# Save Loss plot
plt.figure()
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title("Loss")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.savefig('loss_plot_bilstm.png')  # Save the plot as a PNG file
plt.close()

# Save Accuracy plot
plt.figure()
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title("Accuracy")
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.savefig('accuracy_plot_bilstm.png')  # Save the plot as a PNG file
plt.close()

# Save Precision plot
plt.figure()
plt.plot(history.history['precision'], label='Train Precision')
plt.plot(history.history['val_precision'], label='Validation Precision')
plt.legend()
plt.title("Precision")
plt.xlabel('Epochs')
plt.ylabel('Precision')
plt.savefig('precision_plot_bilstm.png')  # Save the plot as a PNG file
plt.close()

# Save Recall plot
plt.figure()
plt.plot(history.history['recall'], label='Train Recall')
plt.plot(history.history['val_recall'], label='Validation Recall')
plt.legend()
plt.title("Recall")
plt.xlabel('Epochs')
plt.ylabel('Recall')
plt.savefig('recall_plot_bilstm.png')  # Save the plot as a PNG file
plt.close()

# Save F1 Score plot
plt.figure()
plt.plot(history.history['f1_score'], label='Train F1 Score')
plt.plot(history.history['val_f1_score'], label='Validation F1 Score')
plt.legend()
plt.title("F1 Score")
plt.xlabel('Epochs')
plt.ylabel('F1 Score')
plt.savefig('f1_score_plot_bilstm.png')  # Save the plot as a PNG file
plt.close()


In [14]:
bi_lstm_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten (Flatten)                    │ (None, 12288)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape (Reshape)                    │ (None, 16, 768)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 128)                 │         426,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 6)                   │             774 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,331,348 (5.08 MB)

 Trainable params: 443,782 (1.69 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 887,566 (3.39 MB)